## Import required libraries

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [2]:
pip install scikit-image


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: /opt/homebrew/Caskroom/miniforge/base/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip list

Package                                           Version
------------------------------------------------- -----------
absl-py                                           1.3.0
aiohttp                                           3.8.3
aiosignal                                         1.3.1
altair                                            4.2.2
appnope                                           0.1.3
asttokens                                         2.2.1
astunparse                                        1.6.3
async-timeout                                     4.0.2
attrs                                             22.1.0
backcall                                          0.2.0
beautifulsoup4                                    4.11.1
blinker                                           1.5
brotlipy                                          0.7.0
bs4                                               0.0.1
cached-property                                   1.5.2
cachetools                              

pyobjc-framework-SharedWithYou                    9.0.1
pyobjc-framework-SharedWithYouCore                9.0.1
pyobjc-framework-ShazamKit                        9.0.1
pyobjc-framework-Social                           9.0.1
pyobjc-framework-SoundAnalysis                    9.0.1
pyobjc-framework-Speech                           9.0.1
pyobjc-framework-SpriteKit                        9.0.1
pyobjc-framework-StoreKit                         9.0.1
pyobjc-framework-SyncServices                     9.0.1
pyobjc-framework-SystemConfiguration              9.0.1
pyobjc-framework-SystemExtensions                 9.0.1
pyobjc-framework-ThreadNetwork                    9.0.1
pyobjc-framework-UniformTypeIdentifiers           9.0.1
pyobjc-framework-UserNotifications                9.0.1
pyobjc-framework-UserNotificationsUI              9.0.1
pyobjc-framework-VideoSubscriberAccount           9.0.1
pyobjc-framework-VideoToolbox                     9.0.1
pyobjc-framework-Virtualization                 

## Store data, labels in the list

In [2]:
data = []
labels = []
# We have 43 Classes
classes = 43
cur_path = os.getcwd()

In [3]:
cur_path

'/Users/prashanthreddy/Documents/Major project/traffic-sign-detection-main'

## Preprocess the images

In [4]:
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '//'+ a)
            image = image.resize((30,30))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except Exception as e:
            print(e)

cannot identify image file '/Users/prashanthreddy/Documents/Major project/traffic-sign-detection-main/train/6//.DS_Store'


## Converting lists into numpy arrays

In [5]:
data = np.array(data)
labels = np.array(labels)

## Save Labels & Data for future use

In [6]:
# os.mkdir('training')
np.save('./training/data',data)
np.save('./training/target',labels)

## Load data & Labels

In [7]:
data=np.load('./training/data.npy')
labels=np.load('./training/target.npy')

In [8]:
print(data.shape, labels.shape)

(39209, 30, 30, 3) (39209,)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=0)

In [10]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(31367, 30, 30, 3) (7842, 30, 30, 3) (31367,) (7842,)


## Convert labels to onehot encoding

In [11]:
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

## Now it's time to build the model

In [12]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
# We have 43 classes that's why we have defined 43 in the dense
model.add(Dense(43, activation='softmax'))

Metal device set to: Apple M1


2023-04-14 12:24:55.637440: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-14 12:24:55.637612: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [13]:
#Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 20
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/20


2023-04-14 12:24:56.486195: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-14 12:24:56.857372: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


981/981 [==============================] - ETA: 0s - loss: 1.8535 - accuracy: 0.5454

2023-04-14 12:25:39.327970: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


981/981 [==============================] - 50s 44ms/step - loss: 1.8535 - accuracy: 0.5454 - val_loss: 0.4524 - val_accuracy: 0.8996
Epoch 2/20
309/981 [========>.....................] - ETA: 17s - loss: 0.6620 - accuracy: 0.8074

In [ ]:
model.summary()

##  Visualising a Model

In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=True,
    dpi=96,
    layer_range=None,
    show_layer_activations=True,
)

In [ ]:
# accuracy 
plt.figure(0)
plt.plot(history.history['accuracy'], label='training accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
# Loss
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

## Let's do testing on Test data

In [ ]:
def testing(testcsv):
    y_test = pd.read_csv(testcsv)
    label = y_test["ClassId"].values
    imgs = y_test["Path"].values
    data=[]
    for img in imgs:
        image = Image.open(img)
        image = image.resize((30,30))
        data.append(np.array(image))
    X_test=np.array(data)
    return X_test,label

In [ ]:
X_test, label = testing('Test.csv')

In [ ]:
Y_pred = model.predict(X_test)

Y_pred= np.argmax(Y_pred,axis=1)
Y_pred

## Accuracy with the test data

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(label, Y_pred))

## Save the model

In [ ]:
model.save("./training/TSR.h5")

## Load the Model

In [ ]:
import os
# os.chdir(r'D:\traffic')
from keras.models import load_model
model = load_model('./training/TSR.h5')

In [ ]:
# Classes of trafic signs
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians',  
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
def test_on_img(img):
    data=[]
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))
    X_test=np.array(data)
    predict_x = model.predict(X_test)
    classes_x=np.argmax(predict_x,axis=1)
    Y_pred = classes[int(classes_x)]
    return image,Y_pred

In [ ]:
plot,prediction = test_on_img(r'/Users/prashanthreddy/Documents/Major project/traffic-sign-detection-main/Test/00001.png')
 
print("Predicted traffic sign is: ", prediction)
plt.imshow(plot)
plt.show()